<a href="https://colab.research.google.com/github/Luizcarlosqueiroz/Otimizacao_por_Enxame_de_Particulas/blob/main/Otimiza%C3%A7%C3%A3o_por_Enxame_de_Part%C3%ADculas_Rosenbrock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importar as Bibliotecas Básicas

In [1]:
import random
import math
import numpy as np 

## Função Objetivo (Rosenbrock)
$$f(x_1 \cdots x_n) = \sum_{i=1}^{n-1} (100(x_i^2 - x_{i+1})^2 + (1-x_i)^2)$$ 
$$-2.048 \leq x_i \leq 2.048$$ 
$$\text{minimum at }f(1, 1, \cdots, 1) = 0$$ 

In [2]:
#https://www.cs.unm.edu/~neal.holts/dga/benchmarkFunction/rosenbrock.html

def funcao_objetivo(X):
  x, y = X
  return (1-x)**2 + 100. * (y-x**2)**2

### Posição Inicial e Atualização

In [3]:
def initPosition(Np, Nd, xMin, xMax):
  return xMin + np.random.rand(Np, Nd)*(xMax-xMin)


In [4]:
def updatePosition (R, V, Np, Nd, xMin, xMax):

  R += V

  # Caso a particula fuja do domínio de busca desejado
  for particle in range(Np):
    for dimension in range(Nd):
      if R[particle][dimension] > xMax:
        R[particle][dimension] = xMax
      if R[particle][dimension] < xMin:
        R[particle][dimension] = xMin
        
  return R

### Velocidade e Atualização

In [5]:
def initVelocity(Np, Nd, vMin, vMax):
    V = vMin + np.random.rand(Np, Nd)*(vMax-vMin)
    return V

In [6]:
def updateVelocity(R, V, Np, Nd, w, c1, c2, vMin, vMax, chi, pBestPos, gBestPos):
    
    r1 = np.random.rand()
    r2 = np.random.rand()

    for particle in range(Np):
        V[particle, :] = chi * (w * V[particle, :] + r1*c1*(pBestPos[particle, :] - R[particle, :]) + r2*c2*(gBestPos - R[particle, :]))

        for dimension in range(Nd):
            if V[particle][dimension] > vMax: 
                V[particle][dimension] = vMax
            if V[particle][dimension] < vMin: 
                V[particle][dimension] = vMin

    return V

In [7]:
def updateFitness(R, Np, gBestValue, gBestPos, pBestValue, pBestPos):

    for particle in range(Np):
        M = funcao_objetivo(R[particle, :])

        if M < gBestValue:
            gBestValue = M
            gBestPos = R[particle, :]

        if M < pBestValue[particle]:
            pBestValue[particle] = M
            pBestPos[particle, :] = R[particle, :]

    return gBestValue, gBestPos, pBestValue, pBestPos

## Código Final

In [8]:
lista_de_gbests = []

for _ in range(30):

  #Número de particulas, Número de dimensões, Número de Iterações
  Np, Nd, Nt = 20, 2, 100
  #Pesos para atualizar a velocidade (seguindo o padrão)
  c1, c2 = 2.05 , 2.05
  #Definição do min e max do coeficiente de inércia
  wMin, wMax = 0.4, 0.9

  #Espaço (de acordo com os limites dados pela função de Rastring)
  xMin, xMax = -5.12 , 5.12
  #Velocidades (imaginando que as particulas podem se mover menos, dado um espaço amostral menor)
  vMin, vMax = 0.15*xMin , 0.15*xMax

  #Visto que o problema é de minimização, inicializando os best's com valor muito altos
  gBestValue = float("inf")
  pBestValue = [float("inf")] * Np

  pBestPos = np.zeros((Np, Nd))
  gBestPos = np.zeros(Nd)

  #Calculando o fator de Clerck
  #Considerando phi > 4, o chi será de 0.7, fazendo com que a V esteja sempre diminuindo
  phi = c1 + c2
  chi = 2.0/abs(2.0-phi-np.sqrt(pow(phi, 2)-4*phi))

  #Inicializando as particulas e velocidade
  R = initPosition(Np, Nd, xMin, xMax)
  V = initVelocity(Np, Nd, vMin, vMax)

  history = []

  for j in range (0, Nt):

    R = updatePosition(R, V, Np, Nd, xMin, xMax)

    gBestValue, gBestPos, pBestValue, pBestPos = updateFitness(R, Np, gBestValue, gBestPos, pBestValue, pBestPos)
    history.append(gBestValue)

    w = wMax - ((wMax-wMin)/Nt)*j
    V = updateVelocity(R, V, Np, Nd, w, c1, c2, vMin, vMax, chi, pBestPos, gBestPos)

  lista_de_gbests.append(gBestValue)

In [9]:
lista_de_gbests

[6.834513488983706e-21,
 4.700556575528635e-19,
 2.718869103682414e-20,
 8.94862198179513e-19,
 1.2914633068319704e-19,
 1.4894718525005007e-20,
 6.068524499846889e-19,
 4.413158570043958e-18,
 7.063549647693548e-21,
 8.103411466032567e-21,
 1.207692443904195e-22,
 2.8993761870482033e-16,
 6.783973227042864e-19,
 2.7177764035169902e-17,
 1.1377186863362342e-22,
 2.5233298860067707e-18,
 1.5236650353920955e-17,
 2.935312593624044e-19,
 8.160184164272163e-21,
 5.1786191952799035e-21,
 6.1301643360271484e-21,
 4.968581976120647e-21,
 6.192602676626088e-24,
 1.0989926942611979e-15,
 4.082560759214819e-17,
 2.355222243485286e-24,
 8.374439341895024e-19,
 1.7627440435855826e-20,
 4.206629828986334e-19,
 1.0930658128925906e-18]